In [ ]:
import os
import gc
import cv2
import sys
import math
import time
import copy
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
Path.ls = lambda x: list(x.iterdir())

import albumentations
from albumentations.pytorch import ToTensor, ToTensorV2

import torch
from torch import nn, optim
from torchvision import transforms, models
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

effnet_path = '../input/efficientnet-pytorch/'
sys.path.append(effnet_path)
from efficientnet_pytorch import EfficientNet

test_path = Path("../input/cassava-leaf-disease-classification/test_images")
test_fnames = test_path.ls()

This is the inference notebook. See the [training notebook](https://www.kaggle.com/moeinshariatnia/pytorch-better-normalization-onecycle-lr-train) if that is what you are interested in.

In [ ]:
test_df = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")
train_df = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
num_classes = train_df['label'].nunique()

In [ ]:
mean = [0.4589, 0.5314, 0.3236]
std = [0.2272, 0.2297, 0.2200]

test_tfms = albumentations.Compose([
    albumentations.RandomResizedCrop(256, 256),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.HueSaturationValue(
        hue_shift_limit=0.2, 
        sat_shift_limit=0.2,
        val_shift_limit=0.2, 
        p=0.5
    ),
    albumentations.RandomBrightnessContrast(
        brightness_limit=(-0.1,0.1), 
        contrast_limit=(-0.1, 0.1), 
        p=0.5
    ),
    albumentations.Normalize(
        mean=mean, 
        std=std, 
        max_pixel_value=255.0, 
        p=1.0
    ),
    ToTensorV2()], p=1.)

class LeafDataTest(Dataset):
    def __init__(self, df, split="test"):
        self.transforms = test_tfms
        self.paths = [test_path/id_ for id_ in df['image_id'].values]
    
    def __getitem__(self, idx):
        img = cv2.imread(str(self.paths[idx]))[..., ::-1]
        img = self.transforms(image=img)['image']
       
        return img
    
    def __len__(self):
        return len(self.paths)

def make_dataloaders(batch_size=32, num_workers=4, pin_memory=True, **kwargs):
    dataset = LeafDataTest(**kwargs)
    dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers,
                            pin_memory=pin_memory, shuffle=True if kwargs['split'] == "train" else False)
    return dataloader

In [ ]:
test_dl = make_dataloaders(df=test_df, split="test")

In [ ]:
class EfficientNetModel(nn.Module):
    def __init__(self, arch="b4", dropout=0.2, n_out=5, 
                 pretrained=True, freeze=True):
        super().__init__()
        if pretrained:
            self.model = EfficientNet.from_pretrained(f"efficientnet-{arch}")
            if freeze:
                for p in self.model.parameters():
                    p.requires_grad = False
        else:
            self.model = EfficientNet.from_name(f"efficientnet-{arch}")
        
        self.lin1 = nn.Linear(1792 * 2, 512)
        self.lin2 = nn.Linear(512, n_out)
        self.bn1 = nn.BatchNorm1d(1792 * 2)
        self.bn2 = nn.BatchNorm1d(512)
        self.dropout = dropout
        
    def forward(self, x):
        x = self.model.extract_features(x)
        avg = F.adaptive_avg_pool2d(x, 1)
        max_ = F.adaptive_max_pool2d(x, 1)
        cat = torch.cat((avg.view(x.size(0), -1), max_.view(x.size(0), -1)), dim=1)
        x = self.bn1(cat)
        x = F.dropout(x, self.dropout)
        x = F.relu(self.bn2(self.lin1(x)))
        x = self.lin2(x)
        return x

Loading the saved weights

In [ ]:
model = model = EfficientNetModel(pretrained=False, freeze=False).to(device)
model.load_state_dict(torch.load("../input/pytorch-better-normalization-onecycle-lr-train/effnet.pt", map_location=device))
model.eval();

This goes through the whole test set and concatenates all the predictions.

In [ ]:
def inference_one_pass(model, test_dl):
    model.eval()
    all_preds = []
    with torch.no_grad():
        for batch in tqdm(test_dl):
            preds = model(batch.to(device))
            all_preds.append(preds)
            
    return torch.cat(all_preds, dim=0)

We are doing TTA (test time augmentation). Each images is seen 5 times and finally we take the average of all the predictions.

In [ ]:
num_passes = 5
tta = None
for _ in range(num_passes):
    all_preds = inference_one_pass(model, test_dl)
    if tta is None:
        tta = all_preds
    else:
        tta += all_preds
tta /= float(num_passes)
label_preds = tta.argmax(dim=1)

In [ ]:
test_df['label'] = label_preds.cpu().numpy()

In [ ]:
test_df.to_csv("submission.csv", index=False)